In [6]:
import os
import pickle

import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pygmo as pg
# import seaborn as sns
import tellurium as te
import itertools

os.chdir('..')

from mpl_toolkits.axes_grid1 import make_axes_locatable
from src.odbm.odbm_main import ModelBuilder
from src.fitting.pygmo_problems import SBML_Barebone_Multi_Fly

Iterate through all factor labels for enzymes and metabolites. Condense into one variable called parameter_labels

In [11]:
pvals = {p:r_simple[p] for p in r_simple.getGlobalParameterIds() if 'p_EC' in p}
plabels = list(pvals.keys())

metabolites = [s for s in r_simple.getFloatingSpeciesIds() if 'EC' not in s]
rstr = r_simple.getSBML()
vlabels = [v for v in [(rstr[rstr.find('<initialAssignment symbol="'+s+'">')+150:rstr.find('<initialAssignment symbol="'+s+'">')+150+len(s)]).split(' ')[0] for s in metabolites] if 'v' in v]

parameter_labels = plabels + vlabels

In [12]:
parameter_labels

['p_EC11137', 'v338', 'v581', 'v359', 'v593', 'v539']

Makes a dataframe of all possible perturbations of starting conditions of enzymes and metabolites in the system. These are the initial conditions we will want to break down into numpy arrays for "x".

In [13]:
# Example list of IDs and perturbation factors
IDs = parameter_labels
perturbation_factors = [1/10, 10]

# Initialize empty dataframe to store results
concentration_df = pd.DataFrame(columns=['Perturbation ID'] + IDs)

# Generate all combinations of perturbation factors for the IDs
perturbation_combinations = list(itertools.product(perturbation_factors, repeat=len(IDs)))

perturbation_id = 1
for combination in perturbation_combinations:
    # Track the concentration values for this perturbation
    concentration_values = [perturbation_id]
    for i, ID in enumerate(IDs):
        initial_concentration = r_simple.getValue(ID)
        perturbed_concentration = initial_concentration * combination[i]
        concentration_values.append(perturbed_concentration)
    concentration_df.loc[len(concentration_df)] = concentration_values
    perturbation_id += 1

In [14]:
concentration_df

,Perturbation ID,p_EC11137,v338,v581,v359,v593,v539
0,1.0,0.1,0.1,0.1,0.1,0.1,0.1
1,2.0,0.1,0.1,0.1,0.1,0.1,10.0
2,3.0,0.1,0.1,0.1,0.1,10.0,0.1
3,4.0,0.1,0.1,0.1,0.1,10.0,10.0
4,5.0,0.1,0.1,0.1,10.0,0.1,0.1
...,...,...,...,...,...,...,...
59,60.0,10.0,10.0,10.0,0.1,10.0,10.0
60,61.0,10.0,10.0,10.0,10.0,0.1,0.1
61,62.0,10.0,10.0,10.0,10.0,0.1,10.0
62,63.0,10.0,10.0,10.0,10.0,10.0,0.1


Calling SBML_Barebone_Multi_Fly. We'll want the input for calculate_metrics to be the numpy array we got from the above dataframe before running though.

In [25]:
mod = ['src/frenda_brenda/Files/SBML models/240610MC_simple.sbml']
parameter_labels = parameter_labels
tp = 60 # try one hour to start with
# var = [concentration_df.iloc[0].drop(concentration_df.columns[0]).to_dict()]

In [32]:
peter = SBML_Barebone_Multi_Fly(mod, parameter_labels, tp)
peter._setup_rr()
metrics = peter._calculate_metrics(np.array([1,1,1,1,1,1]))

In [36]:
metrics

[    Species  Final Concentration  Min Conc  Max Conc   Min Time   Max Time  \
 0  [C00003]             0.000316  0.000316  0.001000  36.096096   0.000000   
 1  [C00004]             1.083684  1.083000  1.083684   0.000000  36.096096   
 2  [C00036]             0.000684  0.000000  0.000684   0.000000  36.096096   
 3  [C00080]             0.000784  0.000100  0.000784   0.000000  36.096096   
 4  [C00149]             1.699316  1.699316  1.700000  36.096096   0.000000   
 
    Total Production  Production to Max  Half Max Time  Half Max Conc  
 0         -0.000684           0.000000       0.000000       0.001000  
 1          0.000684           0.000684       1.201201       1.083339  
 2          0.000684           0.000684       1.201201       0.000339  
 3          0.000684           0.000684       1.201201       0.000439  
 4         -0.000684           0.000000       0.000000       1.700000  ]